# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770723729957                   -0.53    9.0         
  2   -2.772139017895       -2.85       -1.30    1.0    115ms
  3   -2.772170168083       -4.51       -2.63    1.0    137ms
  4   -2.772170721793       -6.26       -4.02    2.0    123ms
  5   -2.772170722984       -8.92       -4.76    2.0    147ms
  6   -2.772170723014      -10.54       -5.75    1.0    111ms
  7   -2.772170723015      -11.84       -6.21    2.0    128ms
  8   -2.772170723015      -13.35       -7.10    1.0    130ms
  9   -2.772170723015   +  -14.03       -8.19    1.0    116ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770708846978                   -0.53    8.0         
  2   -2.772050698551       -2.87       -1.31    1.0    106ms
  3   -2.772082837847       -4.49       -2.61    1.0    124ms
  4   -2.7

1.7735580927986587

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770749865326                   -0.52    9.0         
  2   -2.772059341424       -2.88       -1.32    1.0    103ms
  3   -2.772083130966       -4.62       -2.48    1.0    131ms
  4   -2.772083344826       -6.67       -3.18    1.0    107ms
  5   -2.772083415617       -7.15       -3.85    2.0    157ms
  6   -2.772083417666       -8.69       -4.64    1.0    110ms
  7   -2.772083417808       -9.85       -5.22    2.0    148ms
  8   -2.772083417810      -11.61       -6.19    1.0    114ms
  9   -2.772083417811      -12.79       -6.63    2.0    136ms
 10   -2.772083417811      -13.91       -7.66    1.0    122ms
 11   -2.772083417811   +  -14.15       -8.21    1.0    126ms

Polarizability via ForwardDiff:       1.7725349764199978
Polarizability via finite difference: 1.7735580927986587
